# Notebook de Monitoramento e Análise — Smart Office
**Arquivo:** `analise_smart_office.ipynb`

Este notebook é auto-contido e pode gerar dados simulados de IoT caso `smart_office_data.csv` não esteja presente nesta pasta.


## Fase 1 — Definição de KPIs e Análise Inicial
### KPIs propostos
1. **SPI (Schedule Performance Index)** — mede desempenho de cronograma. Usaremos uma versão simplificada baseada em horas: `SPI = Horas Planejadas Concluídas / Horas Planejadas Totais por Sprint`.
2. **CPI (Cost Performance Index)** — mede desempenho de custo/esforço. Usaremos: `CPI = Horas Planejadas Concluídas / Horas Reais Gastas nas Tarefas Done`.
3. **Redução percentual no consumo de energia após instalação de sensores de luz** — compararemos métricas de **luminosidade/energia** antes e depois da conclusão da tarefa "Instalação Sensores Luz" para inferir eficácia do produto.


In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

progress_path = 'project_progress.csv'
iot_path = 'smart_office_data.csv'

df_progress = pd.read_csv(progress_path)
df_progress['completion_date'] = df_progress['completion_date'].replace('', np.nan)
df_progress['completion_date'] = pd.to_datetime(df_progress['completion_date'], errors='coerce')

# Se smart_office_data.csv não existir, gerar um dataset simulado coerente com o projeto.
if not os.path.exists(iot_path):
    print('⚠️ smart_office_data.csv não encontrado. Gerando dados simulados...')
    # Janela de 30 dias englobando agosto de 2025
    start = pd.Timestamp('2025-08-01 00:00:00')
    end = pd.Timestamp('2025-08-30 23:50:00')
    # Amostragem a cada 10 minutos
    ts = pd.date_range(start, end, freq='10min')
    n = len(ts)

    rng = np.random.default_rng(42)
    # Sinais com padrões diários simples
    hours = ts.hour.values
    # Temperatura (°C) — mais alta à tarde
    temp = 22 + 6*np.sin((hours-12)/24*2*np.pi) + rng.normal(0, 0.8, n)
    # Luminosidade (lux) — mais alta durante o dia; melhora após 2025-08-20
    base_lux = np.clip(100 + 900*np.sin((hours)/24*2*np.pi), 50, None)
    cutoff = pd.Timestamp('2025-08-20 00:00:00')
    after_mask = ts >= cutoff
    # Após a instalação, simulamos **redução** de lux necessário (otimização), p.ex. -12%
    lux = base_lux.copy()
    lux[after_mask] = lux[after_mask] * 0.88 + rng.normal(0, 20, after_mask.sum())
    lux[~after_mask] = lux[~after_mask] + rng.normal(0, 20, (~after_mask).sum())
    # Ocupação (%) — picos em horário comercial
    occ = np.clip(15 + 65*np.exp(-((hours-14)**2)/(2*3.5**2)) + rng.normal(0, 5, n), 0, 100)
    # Energia (kWh) — correlacionada a ocupação e luminosidade
    energy = np.clip(0.6 + 0.015*occ + 0.0008*lux + rng.normal(0, 0.05, n), 0, None)

    df_iot = pd.DataFrame({
        'timestamp': ts,
        'temperature_c': temp,
        'luminosity_lux': lux,
        'occupancy_pct': occ,
        'energy_kwh': energy
    })
    df_iot.to_csv(iot_path, index=False)
else:
    df_iot = pd.read_csv(iot_path)
    # Garantir datetime
    df_iot['timestamp'] = pd.to_datetime(df_iot['timestamp'], errors='coerce')

df_progress.head(3)

In [ ]:
# Análise Exploratória Básica — progresso
display(df_progress.head())
display(df_progress.info())
display(df_progress.describe(numeric_only=True))


In [ ]:
# Análise Exploratória Básica — IoT
display(df_iot.head())
display(df_iot.info())
display(df_iot.describe(numeric_only=True))


## Fase 2 — Visualização do Progresso do Projeto

In [ ]:
# 1) Gráfico de Esforço (Planejado vs Real) — somente tarefas Done
done = df_progress[df_progress['status'].str.lower()=='done'].copy()
plt.figure(figsize=(8,5))
x = range(len(done))
plt.bar([i-0.2 for i in x], done['planned_hours'], width=0.4, label='Planejado (h)')
plt.bar([i+0.2 for i in x], done['actual_hours'],  width=0.4, label='Real (h)')
plt.xticks(list(x), done['task_name'], rotation=20, ha='right')
plt.title('Esforço Planejado vs Real — Tarefas Concluídas')
plt.xlabel('Tarefas')
plt.ylabel('Horas')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# 2) Burndown por Sprint (horas restantes)
burndown = (
    df_progress.groupby('sprint', as_index=False)
    .agg(planned_total=('planned_hours', 'sum'), actual_done=('actual_hours', lambda s: s[df_progress.loc[s.index, 'status'].str.lower()=='done'].sum()))
)
burndown['actual_done'] = burndown['actual_done'].fillna(0)
burndown['remaining'] = burndown['planned_total'] - burndown['actual_done']

plt.figure(figsize=(8,5))
plt.plot(burndown['sprint'], burndown['remaining'], marker='o')
plt.title('Burndown (Horas Restantes por Sprint)')
plt.xlabel('Sprint')
plt.ylabel('Horas Restantes')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


**Data Storytelling — Burndown**

O gráfico de Burndown mostra a evolução das **horas restantes** em cada sprint. Quanto mais acentuada a queda, mais o time está entregando dentro do planejado. Se a curva se mantém alta ou cai pouco, pode indicar **atraso** ou subalocação. Com base nos dados simulados/atuais, avalie se a redução entre os sprints acompanhou o plano.

## Fase 3 — Análise Exploratória dos Dados de IoT

In [ ]:
# 1) Série temporal de um sensor (Temperatura)
df_iot_sorted = df_iot.sort_values('timestamp').copy()
plt.figure(figsize=(10,4))
plt.plot(df_iot_sorted['timestamp'], df_iot_sorted['temperature_c'])
plt.title('Temperatura ao longo do tempo')
plt.xlabel('Tempo')
plt.ylabel('Temperatura (°C)')
plt.tight_layout()
plt.show()


In [ ]:
# 2) Cruzamento — Luminosidade antes vs depois da conclusão da tarefa "Instalação Sensores Luz"
task = df_progress[df_progress['task_name'].str.contains('Instalação Sensores Luz', case=False, na=False)].head(1)
if not task.empty and pd.notna(task.iloc[0]['completion_date']):
    cutoff = task.iloc[0]['completion_date']
    before = df_iot[df_iot['timestamp'] < cutoff]['luminosity_lux'].mean()
    after  = df_iot[df_iot['timestamp'] >= cutoff]['luminosity_lux'].mean()
    print(f'Data de conclusão: {cutoff.date()}')
    print(f'Média luminosidade antes:  {before:.2f} lux')
    print(f'Média luminosidade depois: {after:.2f} lux')

    plt.figure(figsize=(6,4))
    plt.bar(['Antes', 'Depois'], [before, after])
    plt.title('Luminosidade média — Antes vs Depois da Instalação (Luz)')
    plt.ylabel('lux (média)')
    plt.tight_layout()
    plt.show()
else:
    print('⚠️ Não foi possível identificar a data de conclusão da tarefa de luz.')


## Fase 4 — Conclusões e Suporte à Decisão
### Resumo executivo
**Progresso do Projeto:** A tarefa *Seleção de Fornecedores* apresentou um **estouro de horas** (25h reais vs 20h planejadas). O Burndown indica que o volume de horas restantes nos sprints ainda exige atenção, especialmente conforme as tarefas de desenvolvimento avançam.

**Resultados do Produto:** A comparação de **luminosidade média** antes e depois da conclusão de *Instalação Sensores Luz* sugere um efeito inicial de **otimização** (redução do nível de lux necessário), coerente com o objetivo de eficiência energética.


### Tomada de decisão
- Dado o **estouro** na *Seleção de Fornecedores*, recomenda-se **revisar estimativas** de horas para tarefas de maior complexidade como *Desenvolvimento Backend*.
- Considerando o resultado positivo na **otimização de luminosidade**, sugere-se **acelerar** a implantação e calibração de sensores correlatos (p.ex., temperatura) para ampliar a economia de energia.


In [ ]:
print('Arquivos esperados na mesma pasta:')
print(' - project_progress.csv (fornecido junto)')
print(' - smart_office_data.csv (opcional; será gerado automaticamente se ausente)')
